In [4]:
!pip install fastai

^C


In [1]:
from flask import Flask, render_template, request, redirect, url_for
from werkzeug.utils import secure_filename
import os
import random as rnd
from pathlib import Path
from fastai import *
from fastai.vision.all import *

app = Flask(__name__)
if not os.path.isdir('UploadFolder'): os.mkdir('UploadFolder')
app.config['UPLOAD_FOLDER'] = 'UploadFolder'

# These are the categories used in the model and the data
CATEGORIES = ["art deco","baroque","gothic","roman","tudor"]
# Every category has at least 600 images
N_MAX = 600

def get_examples(model):
   examples = []
   for i in range(4):
      cat = rnd.choice(CATEGORIES)
      n = rnd.choice(range(600))

def get_model():
   pat ='export.pkl'
   model = load_learner(pat)
   print(" * Model loaded!")
   return model
    
model = get_model()

@app.route('/')
def home():
   
   return render_template('home.html',examples=examples)

@app.route("/testImage_post", methods=['post'])
def testImage_post():
   # examples are manualy set for now, straight from the model? or select some to test and test them and than compare to name using regex?

   image = request.files['file']
   imageName = secure_filename(image.filename)
   image.save(os.path.join(app.config['UPLOAD_FOLDER'], imageName))
   #test the posted image with the model
   return redirect(url_for('home'))



if __name__ == '__main__':
   app.run()

NotImplementedError: cannot instantiate 'PosixPath' on your system